In [25]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from numpy import sqrt
from scipy.integrate import solve_ivp

In [26]:
def Lmatrix(u):
    L = np.array([ [u[0], -u[1], -u[2],  u[3]],
                   [u[1],  u[0], -u[3], -u[2]],
                   [u[2],  u[3],  u[0],  u[1]],
                   [u[3], -u[2],  u[1], -u[0]]])
    return L

In [27]:
def ks2rv(ks):
    u = ks[0:4].copy()
    du = ks[4:8].copy()
    L = Lmatrix(u)
    x = L@u
    rvect = x[0:3]
    dx = 2*L@du
    dr = dx[0:3]
    r = norm(rvect)
    vvect = dr/r
    return np.append(rvect, vvect)

In [28]:
def rv2ks(rv):
    rvect = rv[0:3].copy()
    vvect = rv[3:6].copy()
    r = norm(rvect)
    u = np.empty((4,))
    if rvect[0] >= 0:
        u[0] = sqrt((r + rvect[0])/2)
        u[1] = rvect[1]*u[0]/(r + rvect[0])
        u[2] = rvect[2]*u[0]/(r + rvect[0])
        u[3] = 0.0
    else:
        u[1] = sqrt((r - rvect[0])/2)
        u[2] = 0.0
        u[0] = rvect[1]*u[1]/(r - rvect[0])
        u[3] = rvect[2]*u[1]/(r - rvect[0])
    L = Lmatrix(u)
    dr = r*vvect
    du = 1/r/2*L.T@np.append(dr, [0.0])
    return np.append(u, du)

In [20]:
def r2bp(t, y):
    rvect = y[0:3].copy()
    vvect = y[3:6].copy()
    r = norm(rvect)
    dydt = np.append(vvect, -mu*rvect/r**3)
    return dydt

In [45]:
def  r2bp_ks(t, z, h):
    u = z[0:4].copy()
    du = z[4:8].copy()
    r = z[8]
    q = z[9]
    t = z[10]
    dzds = np.empty((11,))
    dzds[0:4] = du
    dzds[4:8] = (h/2)*u
    dzds[8] = q
    dzds[9] = 2*h*r + mu
    dzds[10] = r
    return dzds

In [52]:
def events(s, z, tf):
    return z[10] - tf

In [53]:
t0 = 0.0
tf = 10*2*np.pi
mu = 1.0

rvect0 = np.array([1.0, 0.0, 0.0])
vvect0 = np.array([0.0, 1.0, 0.0])
y0 = np.append(rvect0, vvect0)
r0 = norm(rvect0)
v0 = norm(vvect0)
q0 = np.dot(rvect0, vvect0)
h0 = v0**2/2 - 1.0/r0

ks0 = rv2ks(y0)
s0 = 0.0
sf = 100.0
z0 = np.concatenate((ks0, [r0], [q0], [t0]))

r2bp_ks_r = lambda s, z : r2bp_ks(s, z, h0)
events_r = lambda s, z : events(s, z, tf)
events_r.terminal = True
events_r.direction = +1

In [54]:
tol = 1.0e-6
method = 'RK45'

sol = solve_ivp(r2bp, [t0, tf], y0, atol=tol, rtol=tol, method=method)
sol_ks = solve_ivp(r2bp_ks_r, [s0, sf], z0, atol=tol, rtol=tol, method=method, events=events_r)
rv1 = ks2rv(sol_ks.y[0:8, -1])
rv0 = ks2rv(sol_ks.y[0:8, 0])

print(f"{norm(sol.y[0:3, -1] - sol.y[0:3, 0]):6.6e}")
print(f"{norm(rv1[0:3] - rv0[0:3]):6.6e}")
print(sol.nfev)
print(sol_ks.nfev)

9.794763e-03
5.759005e-05
1244
608
